In [23]:
import pandas as pd

In [24]:
seasons=pd.read_csv('../Data/df_all_seasons_cleaned.csv')
games=pd.read_csv('../Data/df_game_cleaned.csv')
players=pd.read_csv('../Data/df_player_cleaned.csv')
players2=pd.read_csv('../Data/df_common_players_cleaned.csv')
score=pd.read_csv('../Data/df_line_score_cleaned.csv')
team_df=pd.read_csv('../Data/df_team_cleaned.csv')


In [21]:
seasons.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,42,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,34,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,12,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,7,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,22,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97


In [25]:
import pandas as pd

# 2. Préparer games → win_rate par équipe/saison
games['season'] = games['season_id'].apply(lambda x: f"{str(x)[-4:]}-{str(int(str(x)[-4:])+1)[-2:]}")
# extraire home/away → stack pour avoir col. [season, team_abbreviation, wl]
home = games[['season','team_abbreviation_home','wl_home']].rename(
    columns={'team_abbreviation_home':'team','wl_home':'wl'})
away = games[['season','team_abbreviation_away','wl_away']].rename(
    columns={'team_abbreviation_away':'team','wl_away':'wl'})
df_results = pd.concat([home,away], ignore_index=True)
team_wr = (df_results
           .groupby(['season','team'])['wl']
           .value_counts(normalize=True)
           .unstack(fill_value=0)
           .rename(columns={'W':'win_rate'})
           .reset_index()[['season','team','win_rate']]
          )

# 3. Préparer score → moyenne de pts par équipe/saison
# idem : transformer home/away en long, puis merge avec games[['game_id','season']]
score_home = score[['game_id','team_abbreviation_home','pts_home']].rename(
    columns={'team_abbreviation_home':'team','pts_home':'pts'})
score_away = score[['game_id','team_abbreviation_away','pts_away']].rename(
    columns={'team_abbreviation_away':'team','pts_away':'pts'})
score_long = pd.concat([score_home,score_away], ignore_index=True)
score2 = pd.merge(score_long,
                  games[['game_id','season']],
                  on='game_id', how='left')
team_avg_pts = (score2
                .groupby(['season','team'])['pts']
                .mean()
                .reset_index()
                .rename(columns={'pts':'avg_pts'})
               )

# 4. Jointure principale : seasons + players2 (pour person_id+position)
df = pd.merge(
    seasons,
    players2[['display_first_last','team_abbreviation','person_id','position']],
    left_on=['player_name','team_abbreviation'],
    right_on=['display_first_last','team_abbreviation'],
    how='left'
)

# 5. Ajouter l’identité détaillée du joueur (players)
df = pd.merge(
    df,
    players[['id','first_name','last_name','is_active']],
    left_on='person_id',
    right_on='id',
    how='left'
)

# 6. Ajouter les infos équipe (team)
df = pd.merge(
    df,
    team[['abbreviation','full_name','nickname','city']],
    left_on='team_abbreviation',
    right_on='abbreviation',
    how='left'
).rename(columns={
    'full_name':'team_full_name',
    'nickname':'team_nickname',
    'city':'team_city'
})

# 7. Ajouter win_rate et avg_pts
df = pd.merge(df, team_wr,      left_on=['season','team_abbreviation'], right_on=['season','team'], how='left')
df = pd.merge(df, team_avg_pts, left_on=['season','team_abbreviation'], right_on=['season','team'], how='left')

# Le DataFrame df contient désormais, ligne par ligne :
#   • les stats individuelles du joueur (seasons)
#   • son person_id & position (players2)
#   • son identité (players)
#   • les métadonnées de l’équipe (team)
#   • son taux de victoire de l’équipe (team_wr)
#   • la moyenne de points de l’équipe (team_avg_pts)


In [28]:
df.describe()

,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,person_id,id,win_rate,avg_pts
count,12816.000000,12816.000000,12816.000000,12816.000000,12816.000000,12816.000000,12816.000000,12816.000000,12816.000000,12816.000000,12816.000000,12816.000000,12816.000000,3.760000e+03,3.760000e+03,12816.000000,12816.000000
mean,27.049001,200.551316,100.262730,51.264747,8.229760,3.565863,1.828675,-2.118844,0.054087,0.140775,0.184570,0.513837,0.131839,4.494835e+05,4.494835e+05,0.490020,100.838661
std,4.339038,9.109069,12.427861,25.001429,6.011792,2.475488,1.800827,10.604280,0.042485,0.061690,0.052388,0.098829,0.094016,6.718954e+05,6.718954e+05,0.145423,7.555756
min,18.000000,160.020000,60.327736,1.000000,0.000000,0.000000,0.000000,-98.500000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000e+00,2.000000e+00,0.000000,81.804878
25%,24.000000,193.040000,90.718400,31.000000,3.600000,1.800000,0.600000,-6.300000,0.021000,0.096000,0.149000,0.482000,0.066000,1.749000e+03,1.749000e+03,0.388889,95.266667
50%,26.000000,200.660000,99.790240,57.000000,6.700000,3.000000,1.200000,-1.300000,0.041000,0.131000,0.181000,0.525000,0.103000,1.011525e+05,1.011525e+05,0.500000,99.407895
75%,30.000000,208.280000,108.862080,73.000000,11.500000,4.700000,2.400000,3.200000,0.083000,0.179000,0.217000,0.563000,0.179000,2.039940e+05,2.039940e+05,0.595745,106.104167
max,44.000000,231.140000,163.293120,85.000000,36.100000,16.300000,11.700000,95.000000,0.600000,1.000000,0.595000,1.500000,1.000000,1.631323e+06,1.631323e+06,0.857143,119.252874


In [29]:
df.to_csv('../Data/df_all.csv', index=False)

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from itertools import combinations

# 1. Charger le dataset fusionné
df = pd.read_csv("../Data/df_all.csv")

# 2. Filtrer pour la saison cible
season = '2022-23'
df_season = df[df['season'] == season].copy()

# 3. Attribuer une position primaire via la taille (en cm)
df_season['height_cm'] = pd.to_numeric(df_season['player_height'], errors='coerce')
df_season['primary_pos'] = np.where(
    df_season['height_cm'] >= 210, 'C',
    np.where(df_season['height_cm'] >= 200, 'F', 'G')
)

# 4. Features à entraîner
features = ['pts', 'reb', 'ast', 'oreb_pct', 'dreb_pct',
            'usg_pct', 'ts_pct', 'ast_pct', 'net_rating']

# 5. Préparer l'entraînement sur les autres saisons
train = df[df['season'] != season]
top5 = train.groupby(['season', 'team_abbreviation'], group_keys=False) \
            .apply(lambda x: x.nlargest(5, 'pts'))
agg = top5.groupby(['season', 'team_abbreviation'])[features].mean().reset_index()
team_wr = df[['season', 'team_abbreviation', 'win_rate']].drop_duplicates()
train_df = pd.merge(agg, team_wr, on=['season', 'team_abbreviation'], how='left')

X_train = train_df[features]
y_train = train_df['win_rate']
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 6. Recherche de la Dream Team 2022-23 (2 G, 2 F, 1 C)
candidates = {
    'G': df_season[df_season['primary_pos']=='G'].nlargest(10, 'net_rating'),
    'F': df_season[df_season['primary_pos']=='F'].nlargest(10, 'net_rating'),
    'C': df_season[df_season['primary_pos']=='C'].nlargest(5, 'net_rating')
}

best_score = -1
best_lineup = None

for g in combinations(candidates['G'].itertuples(), 2):
    for f in combinations(candidates['F'].itertuples(), 2):
        for c in candidates['C'].itertuples():
            lineup = list(g) + list(f) + [c]
            # **Inclure net_rating dans les features pour la prédiction**
            feats = np.mean([[
                p.pts, p.reb, p.ast,
                p.oreb_pct, p.dreb_pct,
                p.usg_pct, p.ts_pct,
                p.ast_pct, p.net_rating
            ] for p in lineup], axis=0)
            pred = model.predict(feats.reshape(1, -1))[0]
            if pred > best_score:
                best_score, best_lineup = pred, lineup

# 7. Afficher la Dream Team
dream = pd.DataFrame([{
    'player': p.player_name,
    'position': p.primary_pos,
    'pts': p.pts,
    'reb': p.reb,
    'ast': p.ast,
    'oreb_pct': p.oreb_pct,
    'dreb_pct': p.dreb_pct,
    'usg_pct': p.usg_pct,
    'ts_pct': p.ts_pct,
    'ast_pct': p.ast_pct,
    'net_rating': p.net_rating
} for p in best_lineup])
dream['predicted_win_rate'] = best_score

dream


/var/folders/t6/j8g_7ls97b9gdfqfkc7p75q80000gn/T/ipykernel_31656/1569351193.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(5, 'pts'))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.

,player,position,pts,reb,ast,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,net_rating,predicted_win_rate
0,Dereon Seabron,G,0.8,0.2,0.0,0.067,0.000,0.156,0.400,0.000,36.8,0.77454
1,Mac McClung,G,12.5,5.0,4.5,0.073,0.152,0.243,0.563,0.290,19.6,0.77454
2,Josh Minott,F,3.1,1.7,0.3,0.054,0.194,0.176,0.582,0.065,10.4,0.77454
3,Jayson Tatum,F,30.1,8.8,4.6,0.029,0.200,0.319,0.607,0.205,8.5,0.77454
4,Nikola Jokic,C,24.5,11.8,9.8,0.080,0.283,0.263,0.701,0.412,12.5,0.77454
